In [26]:
import numpy as np
from matplotlib import cm, pyplot as plt
import matplotlib.dates as dates
import pandas as pd
from datetime import timedelta,date

import warnings
warnings.filterwarnings("ignore")

In [27]:
data_total_pd = pd.read_csv('data_total.csv',index_col =0)

# 修改索引为datetime格式
data_total_pd['date'] = np.array(data_total_pd.index)
data_total_pd['date'] = pd.to_datetime(data_total_pd['date'])
data_total_pd.set_index("date", inplace=True)

data_total_pd.fillna(0,inplace = True)



# 价格往后移5位
market_close_next = data_total_pd['price_raw'].shift(-5)
market_close_diff = market_close_next - data_total_pd['price_raw'] 

data_total_pd['price_next'] = market_close_next
data_total_pd['price_diff'] = market_close_diff


# 仓位的关键参数，初始值默认为1，每次循环要重新赋值
alpha = np.ones(data_total_pd.shape[0])
data_total_pd['alpha'] = alpha

# 评分，评分初始值默认为0，每次循环要重新赋值
score = np.zeros(data_total_pd.shape[0])
data_total_pd['score'] = score

# 评分，评分初始值默认为0，每次循环要重新赋值
score = np.zeros(data_total_pd.shape[0])
data_total_pd['market_yield'] = np.log(data_total_pd['price_next']/data_total_pd['price_raw'])


# 通过仓位调整后的收益率
data_total_pd['adjust_yield'] = np.zeros(data_total_pd.shape[0])

In [28]:
data_total_pd

,price_raw,state,price_next,price_diff,alpha,score,market_yield,adjust_yield
date,,,,,,,,
2015-01-05,3641.541,bb,3513.576,-127.965,1.0,0.0,-0.035773,0.0
2015-01-06,3641.059,bb,3514.040,-127.019,1.0,0.0,-0.035508,0.0
2015-01-07,3643.790,bb,3502.423,-141.367,1.0,0.0,-0.039569,0.0
2015-01-08,3559.259,bb,3604.121,44.862,1.0,0.0,0.012526,0.0
2015-01-09,3546.723,bb,3635.146,88.423,1.0,0.0,0.024625,0.0
2015-01-12,3513.576,bb,3355.155,-158.421,1.0,0.0,-0.046136,0.0
2015-01-13,3514.040,bb,3396.222,-117.818,1.0,0.0,-0.034103,0.0
2015-01-14,3502.423,bb,3548.885,46.462,1.0,0.0,0.013178,0.0
2015-01-15,3604.121,bb,3567.613,-36.508,1.0,0.0,-0.010181,0.0


In [29]:
def cal_score(data_total_pd,n,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss):
    # 每个日期进行循环比对
    date_operate = data_total_pd.index
    
    score_list = []
    for date_i in date_operate[:]:

        # 状态的pandas数组
        States_pd_current = data_total_pd[(data_total_pd.index <= date_i)&(data_total_pd.index > (date_i - timedelta(days = n)))]

        score_current = 0
        for index, row in States_pd_current.iterrows():
            if row['state'] == "bb":
                score_current += score_BigBonus
            elif row['state'] == "bl":
                score_current += score_BigLoss
            elif row['state'] == "mb":
                score_current += score_MinorBonus
            elif row['state'] == "ml":
                score_current += score_MinorLoss
        score_list.append(score_current)

    data_total_pd['score'] = np.array(score_list)
    return data_total_pd


In [ ]:
# 上限总分值：100分
total_score = 8

# 开始循环，寻找合理参数
# 存储参数的数组
para_list = []
i = 0
for score_BigLoss in range(-8,0):
    for score_BigBonus in range(0,9):
        for score_MinorLoss in range(-8,0):
            for score_MinorBonus in range(0,9):
                for n_windows in range(16,30,2):

                    # 仓位的关键参数，初始值默认为1，每次循环要重新赋值
                    alpha = np.ones(data_total_pd.shape[0])
                    data_total_pd['alpha'] = alpha

                    # 评分，评分初始值默认为0，每次循环要重新赋值
                    score = np.zeros(data_total_pd.shape[0])
                    data_total_pd['score'] = score

                    # 通过仓位调整后的收益率，每次循环要重新赋值
                    data_total_pd['adjust_yield'] = np.zeros(data_total_pd.shape[0])

                    data_total_pd = cal_score(data_total_pd,n_windows,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss)

                    data_total_pd['alpha'] = np.where(data_total_pd['score'] > 0,data_total_pd['score']/total_score,0)

                    
                    data_total_pd['adjust_yield'] =  \
                            np.log( 1 + data_total_pd['alpha'] * data_total_pd['price_diff'] / data_total_pd['price_raw'])

                    # 避免后续出错
                    data_total_pd.fillna(0,inplace = True)

                    # 计算胜率
                    win_rate = \
                        float(sum(data_total_pd['adjust_yield'] > data_total_pd['market_yield'])) / float(len(data_total_pd['adjust_yield']))

                    # print win_rate
                    # print sum(data_total_pd['adjust_yield'])
                    if sum(data_total_pd['adjust_yield']) > sum(data_total_pd['market_yield']):
                        if win_rate > 0.52:
                            para_list.append([n_windows,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss,win_rate,sum(data_total_pd['adjust_yield'])])
                            # 提示                   
                            print "找到参数组：n为%d,score_BigBonus为%d,score_BigLoss为%d,score_MinorBonus为%d,score_MinorLoss为%d，胜率为%f，收益率为%f" \
                                    %(n_windows,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss,win_rate,sum(data_total_pd['adjust_yield']))



找到参数组：n为22,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-8，胜率为0.523438，收益率为3.997517
找到参数组：n为20,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-8，胜率为0.520089，收益率为5.452765
找到参数组：n为22,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-8，胜率为0.530134，收益率为6.476382
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-8，胜率为0.523438，收益率为4.173880
找到参数组：n为26,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-8，胜率为0.522321，收益率为2.391585
找到参数组：n为16,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为7,score_MinorLoss为-8，胜率为0.521205，收益率为5.949087
找到参数组：n为22,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为7,score_MinorLoss为-8，胜率为0.527902，收益率为5.644861
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为7,score_MinorLoss为-8，胜率为0.531250，收益率为2.270672
找到参数组：n为22,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-8，胜率为0.520089，收益率为3.000231
找到参数组：n为20,score_BigBonus为0,

找到参数组：n为28,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为4,score_MinorLoss为-3，胜率为0.524554，收益率为1.867987
找到参数组：n为16,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.520089，收益率为4.411204
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.521205，收益率为3.109390
找到参数组：n为26,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.525670，收益率为4.702101
找到参数组：n为28,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.533482，收益率为3.226966
找到参数组：n为22,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-3，胜率为0.526786，收益率为5.478622
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-3，胜率为0.525670，收益率为3.971004
找到参数组：n为26,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-3，胜率为0.525670，收益率为2.787050
找到参数组：n为28,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-3，胜率为0.525670，收益率为0.228991
找到参数组：n为22,score_BigBonus为0,

找到参数组：n为16,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-7，胜率为0.530134，收益率为2.957826
找到参数组：n为18,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-7，胜率为0.527902，收益率为6.747721
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-7，胜率为0.523438，收益率为6.356008
找到参数组：n为26,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-7，胜率为0.530134，收益率为0.774231
找到参数组：n为28,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-7，胜率为0.534598，收益率为2.108793
找到参数组：n为22,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为4,score_MinorLoss为-6，胜率为0.522321，收益率为3.815063
找到参数组：n为16,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-6，胜率为0.520089，收益率为5.742961
找到参数组：n为18,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-6，胜率为0.522321，收益率为5.056006
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-6，胜率为0.524554，收益率为4.153954
找到参数组：n为22,score_BigBonus为1,

找到参数组：n为24,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为7,score_MinorLoss为-4，胜率为0.523438，收益率为2.311536
找到参数组：n为26,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为7,score_MinorLoss为-4，胜率为0.521205，收益率为-0.057900
找到参数组：n为16,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.521205，收益率为1.130322
找到参数组：n为18,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.522321，收益率为4.785877
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.521205，收益率为4.374040
找到参数组：n为22,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.525670，收益率为1.350698
找到参数组：n为16,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为3,score_MinorLoss为-3，胜率为0.520089，收益率为5.025516
找到参数组：n为18,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为3,score_MinorLoss为-3，胜率为0.521205，收益率为4.988220
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为3,score_MinorLoss为-3，胜率为0.521205，收益率为4.993333
找到参数组：n为22,score_BigBonus为1

找到参数组：n为18,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为7,score_MinorLoss为-1，胜率为0.523438，收益率为0.451461
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为7,score_MinorLoss为-1，胜率为0.534598，收益率为3.095012
找到参数组：n为22,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为7,score_MinorLoss为-1，胜率为0.531250，收益率为0.865907
找到参数组：n为18,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-1，胜率为0.525670，收益率为0.574964
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-1，胜率为0.533482，收益率为-0.121583
找到参数组：n为20,score_BigBonus为2,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-8，胜率为0.522321，收益率为4.786863
找到参数组：n为22,score_BigBonus为2,score_BigLoss为-8,score_MinorBonus为5,score_MinorLoss为-8，胜率为0.526786，收益率为5.370383
找到参数组：n为16,score_BigBonus为2,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-8，胜率为0.521205，收益率为5.468552
找到参数组：n为18,score_BigBonus为2,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-8，胜率为0.523438，收益率为7.648897
找到参数组：n为20,score_BigBonus为2

...

找到参数组：n为22,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为3,score_MinorLoss为-3，胜率为0.520089，收益率为5.824090
找到参数组：n为16,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为4,score_MinorLoss为-3，胜率为0.520089，收益率为5.804818
找到参数组：n为28,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为4,score_MinorLoss为-3，胜率为0.521205，收益率为2.985577
找到参数组：n为28,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.521205，收益率为3.395373
找到参数组：n为22,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为7,score_MinorLoss为-3，胜率为0.522321，收益率为4.124522
找到参数组：n为20,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为8,score_MinorLoss为-3，胜率为0.523438，收益率为3.422456
找到参数组：n为22,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为8,score_MinorLoss为-3，胜率为0.520089，收益率为-0.061562
找到参数组：n为20,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为6,score_MinorLoss为-2，胜率为0.520089，收益率为3.858777
找到参数组：n为16,score_BigBonus为4,score_BigLoss为-5,score_MinorBonus为7,score_MinorLoss为-2，胜率为0.520089，收益率为3.591760
找到参数组：n为18,score_BigBonus为4

找到参数组：n为20,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为7,score_MinorLoss为-7，胜率为0.524554，收益率为7.973632
找到参数组：n为22,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为7,score_MinorLoss为-7，胜率为0.526786，收益率为6.121046
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为7,score_MinorLoss为-7，胜率为0.521205，收益率为2.922454
找到参数组：n为26,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为7,score_MinorLoss为-7，胜率为0.525670，收益率为0.168662
找到参数组：n为16,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-7，胜率为0.525670，收益率为2.796887
找到参数组：n为18,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-7，胜率为0.526786，收益率为6.764476
找到参数组：n为20,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-7，胜率为0.524554，收益率为6.322139
找到参数组：n为22,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为4,score_MinorLoss为-6，胜率为0.520089，收益率为3.967581
找到参数组：n为20,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为5,score_MinorLoss为-6，胜率为0.523438，收益率为4.001068
找到参数组：n为22,score_BigBonus为0,

找到参数组：n为24,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为3,score_MinorLoss为-3，胜率为0.524554，收益率为3.827607
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为4,score_MinorLoss为-3，胜率为0.522321，收益率为3.045183
找到参数组：n为26,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为4,score_MinorLoss为-3，胜率为0.527902，收益率为4.760616
找到参数组：n为28,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为4,score_MinorLoss为-3，胜率为0.532366，收益率为3.210989
找到参数组：n为16,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.521205，收益率为4.567360
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.521205，收益率为4.002353
找到参数组：n为26,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.525670，收益率为5.611889
找到参数组：n为28,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为5,score_MinorLoss为-3，胜率为0.533482，收益率为4.174480
找到参数组：n为20,score_BigBonus为0,score_BigLoss为-4,score_MinorBonus为6,score_MinorLoss为-3，胜率为0.524554，收益率为4.136755
找到参数组：n为22,score_BigBonus为0,

找到参数组：n为18,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-8，胜率为0.527902，收益率为7.768956
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-8，胜率为0.522321，收益率为7.265993
找到参数组：n为22,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-8，胜率为0.522321，收益率为3.896903
找到参数组：n为26,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-8，胜率为0.523438，收益率为0.220705
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为4,score_MinorLoss为-7，胜率为0.523438，收益率为5.851417
找到参数组：n为22,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为4,score_MinorLoss为-7，胜率为0.522321，收益率为4.454004
找到参数组：n为16,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为5,score_MinorLoss为-7，胜率为0.520089，收益率为6.096583
找到参数组：n为18,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为5,score_MinorLoss为-7，胜率为0.522321，收益率为5.504453
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为5,score_MinorLoss为-7，胜率为0.529018，收益率为4.812869
找到参数组：n为22,score_BigBonus为1,

找到参数组：n为26,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为7,score_MinorLoss为-5，胜率为0.522321，收益率为0.391243
找到参数组：n为16,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-5，胜率为0.532366，收益率为2.299313
找到参数组：n为18,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-5，胜率为0.530134，收益率为6.149219
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-5，胜率为0.525670，收益率为5.739941
找到参数组：n为22,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-5，胜率为0.525670，收益率为2.540304
找到参数组：n为26,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-5，胜率为0.520089，收益率为-0.302966
找到参数组：n为28,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为8,score_MinorLoss为-5，胜率为0.521205，收益率为1.148928
找到参数组：n为20,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为3,score_MinorLoss为-4，胜率为0.521205，收益率为5.655778
找到参数组：n为22,score_BigBonus为1,score_BigLoss为-4,score_MinorBonus为3,score_MinorLoss为-4，胜率为0.524554，收益率为5.124791
找到参数组：n为24,score_BigBonus为1

In [ ]:
para_pd = pd.DataFrame(para_list)
para_pd.columns = ["n_windows",'score_BigBonus','score_BigLoss','score_MinorBonus','score_MinorLoss','胜率','收益率']

para_pd.to_csv("best_para.csv")